*** Read me ***

Sources du projet :
    * Coin market cap API : https://coinmarketcap.com/api/
    * Coin market cap API doc : https://coinmarketcap.com/api/documentation/v1/#section/Quick-Start-Guide

Objectif du projet :
    * Automate API Extraction
    * Appending Data + Extra

In [9]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data) on ne print pas les datas car probleme de capacité du PC
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type (data)

In [21]:
import pandas as pd

#pour voir le maximum de colonnes possibles
pd.set_option('display.max_columns',None)

#pour voir le maximum de lignes possibles
pd.set_option('display.max_rows', None)

In [ ]:
#pour rendre les données plus lisibles

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [31]:
#Automatisation de la mise à jour des prix pour les 15eres cryptomonnaies selon "coin market cap"

def api_runner():

    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data) on ne print pas les datas car probleme de capacité du PC
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    df
    
#Création d'un fichier CSV
    
    #on verifie l'existence d'un fichier 
    if not os.path.isfile(r'C:\Users\jerem\Desktop\Data analyse\Data analyse Bootcamp Project 6_Python\API.csv') :
        #Si le fichier n'existe pas alors on va écrire comme indiqué ci-dessous avec le nom des colones 
        df.to_csv(r'C:\Users\jerem\Desktop\Data analyse\Data analyse Bootcamp Project 6_Python\API.csv',header='column_names')
    
    
    else : #si le fichier existe alors on fait un append à ce qui existe déjà
        df.to_csv(r'C:\Users\jerem\Desktop\Data analyse\Data analyse Bootcamp Project 6_Python\API.csv',mode ='a', header=FALSE)

In [ ]:
import os

from time import time
from time import sleep

for i in range(333) : #sur coinmarcket cap API on ne peut dépasser 333 exécutions par jour
    api_runner()
    print ('API Runner a terminé sa tache')
    sleep(60)
exit()

In [ ]:
df72=pd.read_csv(r'C:\Users\jerem\Desktop\Data analyse\Data analyse Bootcamp Project 6_Python\API.csv')
df72

In [ ]:
df #affichage du dataframe

In [ ]:
#DATA CLEANING 
#On passe de l'ecriture scientique à l'eciture classique où tous les chiffres sont visibles

pd.set_option('display.floar_format',lambda x : '%.5f' % x)

In [ ]:
#Tri des données en fonction du nom pour pouvoir voir leur évolution dans le temps

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
df5.count()

In [ ]:
#En raison de la structure nous avons besoin d'un index
#creation d'un range et transformer la "serie" en "dataframe"


index = pd.Index(range(90))

# Utilisation de la fonction set_index() pour indexé de la bonne manière
df6 = df5.set_index(index)
df6

# Si le dataframe a l'incide et la valeur il faut utiliser "df5.reset_index()"

In [ ]:
# Changement du nom des colones

df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
#Dans le but de rendre le visuel plus lisible on change le nom des abscisses

df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['24h','7d','30d','60d','90d'])
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
# Simplification du visuel avec un dataframe avec la colonne de notre choix et on tri avec "bitcoin"

df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df10)